In [2]:
import pandas as pd
import numpy as np

In [3]:
cig_data = pd.read_csv("/Users/ellenwu/homework3_attempt1-/data/input/The_Tax_Burden_on_Tobacco__1970-2019_20250304.csv")
cpi_data = pd.read_excel("/Users/ellenwu/homework3_attempt1-/data/input/SeriesReport-20250304165118_4f02a8.xlsx", skiprows=11)

/Users/ellenwu/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
#cleaning tobacco data 

cig_data["measure"] = cig_data["SubMeasureDesc"].map({
    "Average Cost per pack": "cost_per_pack",
    "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
    "Federal and State tax as a Percentage of Retail Price": "tax_percent",
    "Federal and State Tax per pack": "tax_dollar",
    "Gross Cigarette Tax Revenue": "tax_revenue",
    "State Tax per pack": "tax_state"
})
cig_data = cig_data.rename(columns={
    "LocationAbbr": "state_abb", 
    "LocationDesc": "state", 
    "Data_Value": "value"
})
cig_data = cig_data[["state_abb", "state", "Year", "value", "measure"]]

In [5]:
final_data = cig_data.pivot(index=["state", "Year"], columns="measure", values="value").reset_index()
final_data = final_data.sort_values(by=["state", "Year"])

In [6]:
# cleaning CPI data
print("Columns in DataFrame after skipping rows:", cpi_data.columns.tolist()) 

cpi_data = cpi_data.melt(id_vars=["Year"], 
                          value_vars=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                          var_name="month", 
                          value_name="index")

cpi_data = cpi_data.groupby("Year", as_index=False).agg({"index": "mean"})

Columns in DataFrame after skipping rows: ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'HALF1', 'HALF2']


In [7]:
# Adjust to 2010 dollars
final_data = final_data.merge(cpi_data, on='Year', how='outer')  # Use 'outer' to keep all years
final_data["price_cpi"] = final_data["cost_per_pack"] * (218 / final_data["index"])
print(final_data[['Year']].drop_duplicates().sort_values('Year'))

      Year
2550  1913
2551  1914
2552  1915
2553  1916
2554  1917
...    ...
2608  2021
2609  2022
2610  2023
2611  2024
2612  2025

[113 rows x 1 columns]


In [8]:
final_data.to_csv("/Users/ellenwu/homework3/data/output/ftaxburden_data.csv")